In [37]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [41]:
player_df = pd.read_csv('Master/data/Scoring_Agg.csv')
player_df.head()

,playerID,firstName,lastName,HOF,year_first,year_last,seasons,GP,G,A,...,PostA,PostPts,PostPIM,Post+/-,PostPPG,PostPPA,PostSHG,PostSHA,PostGWG,PostSOG
0,aaltoan01,Antti,Aalto,0,1997,2000,4,151,11,17,...,0,0,2,0,0,0,0,0,0,0
1,abbotge01,George,Abbott,0,1943,1943,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,abbotre01,Reg,Abbott,0,1952,1952,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
3,abdelju01,Justin,Abdelkader,0,2007,2011,5,209,18,29,...,2,5,60,-6,0,0,0,0,0,47
4,abelcl01,Clarence,Abel,0,1926,1933,8,333,20,18,...,1,2,58,0,0,0,0,0,0,0


In [42]:
player_df.columns

Index(['playerID', 'firstName', 'lastName', 'HOF', 'year_first', 'year_last',
       'seasons', 'GP', 'G', 'A', 'Pts', 'PIM', '+/-', 'PPG', 'PPA', 'SHG',
       'SHA', 'GWG', 'GTG', 'SOG', 'PostGP', 'PostG', 'PostA', 'PostPts',
       'PostPIM', 'Post+/-', 'PostPPG', 'PostPPA', 'PostSHG', 'PostSHA',
       'PostGWG', 'PostSOG'],
      dtype='object')

In [43]:
df = player_df.drop(['playerID','firstName', 'lastName', 'HOF', 'year_first', 'year_last'], axis=1)
data = player_df.drop(['playerID','firstName', 'lastName', 'HOF', 'year_first', 'year_last'], axis=1)
data.shape
data.dtypes

seasons    int64
GP         int64
G          int64
A          int64
Pts        int64
PIM        int64
+/-        int64
PPG        int64
PPA        int64
SHG        int64
SHA        int64
GWG        int64
GTG        int64
SOG        int64
PostGP     int64
PostG      int64
PostA      int64
PostPts    int64
PostPIM    int64
Post+/-    int64
PostPPG    int64
PostPPA    int64
PostSHG    int64
PostSHA    int64
PostGWG    int64
PostSOG    int64
dtype: object

In [53]:
target = player_df.pop('HOF')
# player_df.dtypes

KeyError: 'HOF'

In [22]:
feature_names = data.columns
# feature_names

Index(['seasons', 'GP', 'G', 'A', 'Pts', 'PIM', '+/-', 'PPG', 'PPA', 'SHG',
       'SHA', 'GWG', 'GTG', 'SOG', 'PostGP', 'PostG', 'PostA', 'PostPts',
       'PostPIM', 'Post+/-', 'PostPPG', 'PostPPA', 'PostSHG', 'PostSHA',
       'PostGWG', 'PostSOG'],
      dtype='object')

In [46]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [48]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

In [52]:
model_func.fit(dict_slices, epochs=100)

Epoch 1/100
429/429 [==============================] - 2s 5ms/step - loss: 0.2030 - accuracy: 0.9698
Epoch 2/100
429/429 [==============================] - 2s 5ms/step - loss: 0.1879 - accuracy: 0.9702
Epoch 3/100
429/429 [==============================] - 3s 7ms/step - loss: 0.1942 - accuracy: 0.9707
Epoch 4/100
429/429 [==============================] - 3s 6ms/step - loss: 0.1876 - accuracy: 0.9702
Epoch 5/100
429/429 [==============================] - 2s 4ms/step - loss: 0.1848 - accuracy: 0.9704
Epoch 6/100
429/429 [==============================] - 5s 13ms/step - loss: 0.1938 - accuracy: 0.9705
Epoch 7/100
429/429 [==============================] - 2s 6ms/step - loss: 0.2004 - accuracy: 0.9693
Epoch 8/100
429/429 [==============================] - 7s 15ms/step - loss: 0.1573 - accuracy: 0.9724
Epoch 9/100
429/429 [==============================] - 8s 19ms/step - loss: 0.1798 - accuracy: 0.9711
Epoch 10/100
429/429 [==============================] - 6s 14ms/step - loss: 0.1816 - ac

In [51]:
model_func.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seasons (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
GP (InputLayer)                 [(None,)]            0                                            
__________________________________________________________________________________________________
G (InputLayer)                  [(None,)]            0                                            
__________________________________________________________________________________________________
A (InputLayer)                  [(None,)]            0                                            
______________________________________________________________________________________________

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
from tensorflow.keras.utils import to_categorical
# One-hot encoding
# y_train_categorical = to_categorical(y_train)
# y_test_categorical = to_categorical(y_test)
y_train_categorical = y_train
y_test_categorical = y_test

In [33]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=26))
model.add(Dense(units=2, activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 6)                 162       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 14        
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100


ValueError: in user code:

    C:\Users\Y243863\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\Y243863\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Y243863\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Y243863\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Y243863\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\U

In [ ]:
deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=2))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [ ]:
deep_model.summary()

In [ ]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Compare the models below

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")